In [66]:
import os
import csv
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

In [67]:
def clean_quotes_semicolon(input_file:str, output_file:str) -> None:
    """
    Clean a CSV file by removing colons, semicolons, and quotes from each field.
    
    Parameters:
        input_file (str): The path to the input CSV file.
        output_file (str): The path to the output CSV file.
    
    Returns:
        None
    """
    data = list()
    with open(input_file, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=';', quotechar='"')

        # Create a new CSV file to write the cleaned data
        with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            for row in reader:
                # Cleaning
                row = [field.replace('"', '').replace("'", "") for field in row]
                row = [field.replace(';', '').replace(',', '.') for field in row]
                
                # Cleaned row write
                writer.writerow(row)

In [68]:
current_path = os.getcwd()
data_path = current_path + '/data'

In [69]:
# Setting up input and output files
files = {
    "caracteristiques":["carcteristiques-{year}.csv", "carcteristiques-{year}-cleaned.csv"],
    "lieux":["lieux-{year}.csv", "lieux-{year}-cleaned.csv"],
    "usagers":["usagers-{year}.csv", "usagers-{year}-cleaned.csv"],
    "vehicules":["vehicules-{year}.csv", "vehicules-{year}-cleaned.csv"]
}

# Run cleaning
year = '2022'
for g_file in files.values():
    input_file = f'{data_path}/2022/{g_file[0].replace("{year}",year)}'  
    output_file = f'{data_path}/cleaned/{year}/{g_file[1].replace("{year}", year)}'    
    clean_quotes_semicolon(input_file=input_file,
                           output_file=output_file)     

In [70]:
caracteristiques = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["caracteristiques"][1].replace("{year}", year)}')
lieux = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["lieux"][1].replace("{year}", year)}')
usagers = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["usagers"][1].replace("{year}", year)}')
vehicules = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["vehicules"][1].replace("{year}", year)}')


/var/folders/fj/25hghm0n7n33b98vlpmy81dm0000gn/T/ipykernel_76113/1448578532.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv(filepath_or_buffer=f'{data_path}/cleaned/{year}/{files["lieux"][1].replace("{year}", year)}')


In [71]:
full_data = caracteristiques.merge(lieux, on="num_acc", how='left').merge(usagers, on='num_acc', how='left').merge(vehicules, on='num_acc', how='left')
full_data.to_csv(f'{data_path}/full_data.csv', index=False)
full_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241487 entries, 0 to 241486
Data columns (total 57 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_acc        241487 non-null  int64  
 1   jour           241487 non-null  int64  
 2   mois           241487 non-null  int64  
 3   an             241487 non-null  int64  
 4   hrmn           241487 non-null  object 
 5   lum            241487 non-null  int64  
 6   dep            241487 non-null  object 
 7   com            241487 non-null  object 
 8   agg            241487 non-null  int64  
 9   int            241487 non-null  int64  
 10  atm            241487 non-null  int64  
 11  col            241487 non-null  int64  
 12  adr            235980 non-null  object 
 13  lat            241487 non-null  float64
 14  long           241487 non-null  float64
 15  catr           241487 non-null  int64  
 16  voie           225068 non-null  object 
 17  v1             241487 non-nul

In [72]:
# Setting up duplicateds using ID as key
full_data_dubs = full_data.duplicated(subset=['num_acc'])
print('joined data total duplicated: %s / %s' % (len(full_data[full_data_dubs]), len(full_data)))
full_data = full_data.drop_duplicates(subset=['num_acc'])
print('Droped duplicates: %s / %s' % (len(full_data[full_data_dubs]), len(full_data)))




joined data total duplicated: 186185 / 241487
joined data total duplicated: 0 / 55302


/var/folders/fj/25hghm0n7n33b98vlpmy81dm0000gn/T/ipykernel_76113/3129052560.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('joined data total duplicated: %s / %s' % (len(full_data[full_data_dubs]), len(full_data)))


In [73]:
from numpy import nan 
full_data = full_data.fillna(nan)
full_data.head()

,num_acc,jour,mois,an,hrmn,lum,dep,com,agg,int,...,id_vehicule_y,num_veh_y,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202200000001,19,10,2022,16:15,1,26,26198,2,3,...,813 952,A01,1,2,0,2,1,9,1,NaN
4,202200000002,20,10,2022,08:34,1,25,25204,2,3,...,813 950,B01,2,7,0,2,8,15,1,NaN
8,202200000003,20,10,2022,17:15,1,22,22360,2,6,...,813 948,A01,2,7,0,2,1,2,1,NaN
12,202200000004,20,10,2022,18:00,1,16,16102,2,3,...,813 947,A01,2,7,0,1,3,16,1,NaN
14,202200000005,19,10,2022,11:45,1,13,13103,1,1,...,813 945,B01,0,7,0,2,4,1,1,NaN


In [74]:
from numpy import nan 
from sklearn.feature_selection import VarianceThreshold


data = merged_df.iloc[:,1:5].values
X = data[:, :-1]
Y = data[:, -1]
print(
    'Shapes', 'X', X.shape, 'Y', Y.shape
)

feature = VarianceThreshold()
# feature.fit_transform(X)

Shapes X (241487, 3) Y (241487,)


In [75]:
new_merge.to_csv(f'{data_path}/cleaned/2005-2021/cleaned.csv', sep=',', index=False)


NameError: name 'new_merge' is not defined